<a href="https://colab.research.google.com/github/Bhavya418/quant_alphas/blob/main/Engulfing_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
import requests
import plotly.graph_objects as go
from datetime import datetime


In [3]:
def fetch_daily_ohlc(symbol, apikey):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={apikey}"
    r = requests.get(url)
    return r.json()


In [4]:
apikey = "SHEYWPL1ZYCDZ17D"
symbol = "RELIANCE.BSE"
data = fetch_daily_ohlc(symbol, apikey)
print(data)


{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'RELIANCE.BSE', '3. Last Refreshed': '2025-04-17', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-04-17': {'1. open': '1238.6500', '2. high': '1279.4500', '3. low': '1227.4000', '4. close': '1274.5500', '5. volume': '392500'}, '2025-04-16': {'1. open': '1239.8500', '2. high': '1240.9000', '3. low': '1229.0500', '4. close': '1238.6500', '5. volume': '361521'}, '2025-04-15': {'1. open': '1256.0000', '2. high': '1256.0000', '3. low': '1237.0000', '4. close': '1239.9000', '5. volume': '719043'}, '2025-04-11': {'1. open': '1200.0000', '2. high': '1222.4500', '3. low': '1197.0500', '4. close': '1219.3000', '5. volume': '684519'}, '2025-04-09': {'1. open': '1172.4000', '2. high': '1189.5000', '3. low': '1169.1000', '4. close': '1185.6000', '5. volume': '495957'}, '2025-04-08': {'1. open': '1177.8500', '2. high': '1196.8000', '3. low': '1163.8500', '4. c

In [5]:
def is_downtrend(candles):
    """True if lows are decreasing = downtrend"""
    low_prices = [float(c["3. low"]) for c in candles]
    for earlier, later in zip(low_prices, low_prices[1:]):
        if earlier <= later:
            return False
    return True

def is_uptrend(candles):
    """True if lows are increasing = uptrend"""
    low_prices = [float(c["3. low"]) for c in candles]

    for earlier, later in zip(low_prices, low_prices[1:]):
        if earlier >= later:
            return False
    return True

In [6]:
def is_bullish_engulfing(prev_candle, current_candle):
    prev_open = float(prev_candle["1. open"])
    prev_close = float(prev_candle["4. close"])
    current_open = float(current_candle["1. open"])
    current_close = float(current_candle["4. close"])

    if prev_close > prev_open or current_close< current_open:
      return False

    return current_open < prev_close and current_close > prev_open

def is_bearish_engulfing(prev_candle, current_candle):
    prev_open = float(prev_candle["1. open"])
    prev_close = float(prev_candle["4. close"])
    current_open = float(current_candle["1. open"])
    current_close = float(current_candle["4. close"])

    if not (prev_close > prev_open and current_close < current_open):
        return False

    return current_open > prev_close and current_close < prev_open


In [11]:
def detect_engulfing(data,lookback_days=3):
    time_series = data.get("Time Series (Daily)", {})
    sorted_dates = sorted(time_series.keys())
    results = []
    for i in range(lookback_days + 1, len(sorted_dates)):
        current_date = sorted_dates[i]
        prev_date = sorted_dates[i - 1]

        current_candle = time_series[current_date]
        prev_candle = time_series[prev_date]
        trend_candles = [time_series[sorted_dates[j]] for j in range(i - lookback_days - 1, i - 1)]
        if is_downtrend(trend_candles) and is_bullish_engulfing(prev_candle, current_candle):
            pattern = "bullish_engulfing"
            entry_price = float(current_candle["4. close"])
            stoploss = min(float(prev_candle["3. low"]), float(current_candle["3. low"]))

        elif is_uptrend(trend_candles) and is_bearish_engulfing(prev_candle, current_candle):
            entry_price = float(current_candle["4. close"])
            stoploss = max(float(prev_candle["2. high"]), float(current_candle["2. high"]))
            pattern = "bearish_engulfing"
        else:
            continue

        results.append({
            "date": current_date,
            "pattern": pattern,
            "entry_price": entry_price,
            "stoploss": stoploss,
            "open": float(current_candle["1. open"]),
            "high": float(current_candle["2. high"]),
            "low": float(current_candle["3. low"]),
            "close": float(current_candle["4. close"]),
            "volume": int(current_candle["5. volume"])
        })

    return pd.DataFrame(results)


In [24]:
def calculate_profit_loss_with_target_and_window(data, engulfing_df, target_pct=2.0, hold_days=3):
    time_series = data.get("Time Series (Daily)", {})
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    trades = []
    total_profit_loss = 0.0

    for _, row in engulfing_df.iterrows():
        entry_date = pd.to_datetime(row["date"])
        direction = row["pattern"]
        entry_price = row["entry_price"]

        if entry_date not in df.index:
            continue

        future_dates = df.index[df.index > entry_date]
        window_dates = future_dates[:hold_days]

        if len(window_dates) == 0:
            continue

        target_price = (
            entry_price * (1 + target_pct / 100)
            if direction == "bullish_engulfing"
            else entry_price * (1 - target_pct / 100)
        )

        exit_price = None
        exit_date = None
        reason = ""

        # Step 1: Check for target hit in window
        for day in window_dates:
            high = df.loc[day]["High"]
            low = df.loc[day]["Low"]
            open_price = df.loc[day]["Open"]

            if direction == "bullish_engulfing" and high >= target_price:
                exit_price = open_price
                exit_date = day
                reason = "target_hit"
                break
            elif direction == "bearish_engulfing" and low <= target_price:
                exit_price = open_price
                exit_date = day
                reason = "target_hit"
                break

        # Step 2: If target not hit, exit at max/min within window
        if exit_price is None:
            if direction == "bullish_engulfing":
                max_day = df.loc[window_dates]["High"].idxmax()
                exit_price = df.loc[max_day]["High"]
                exit_date = max_day
                reason = "max_window_exit"
            elif direction == "bearish_engulfing":
                min_day = df.loc[window_dates]["Low"].idxmin()
                exit_price = df.loc[min_day]["Low"]
                exit_date = min_day
                reason = "min_window_exit"

        if exit_price is None or exit_date is None:
            continue

        profit_loss = (
            exit_price - entry_price if direction == "bullish_engulfing"
            else entry_price - exit_price
        )
        total_profit_loss += profit_loss

        trades.append({
            'entry_date': entry_date,
            'entry_price': entry_price,
            'exit_date': exit_date,
            'exit_price': exit_price,
            'target_price': target_price,
            'direction': direction,
            'profit_loss': profit_loss,
            'exit_reason': reason,
            'stoploss': row.get("stoploss", None)
        })

    return total_profit_loss, trades


In [26]:
def plot_candlestick(data):
    # Extract the time series data and convert it into a DataFrame
    time_series = data.get("Time Series (Daily)", {})

    # Convert the time series into a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Convert all values to float and rename columns for mplfinance compatibility
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    fig.show()

In [25]:
engulfing_df = detect_engulfing(data)
total_profit_loss, trades = calculate_profit_loss_with_target_and_window(
    data,
    engulfing_df,
    target_pct=2.0,
    hold_days=3
)

print(f"Total Profit/Loss: {total_profit_loss:.2f}")
for trade in trades:
    print(
        f"{trade['direction']} | Entry: {trade['entry_date'].date()} at {trade['entry_price']:.2f}, "
        f"Exit: {trade['exit_date'].date()} at {trade['exit_price']:.2f} "
        f"(Target: {trade['target_price']:.2f}, Reason: {trade['exit_reason']}), "
        f"P/L: {trade['profit_loss']:.2f}"
    )


Total Profit/Loss: 58.65
bearish_engulfing | Entry: 2024-12-04 at 1309.00, Exit: 2024-12-09 at 1292.90 (Target: 1282.82, Reason: min_window_exit), P/L: 16.10
bearish_engulfing | Entry: 2025-01-06 at 1218.20, Exit: 2025-01-07 at 1221.40 (Target: 1193.84, Reason: min_window_exit), P/L: -3.20
bullish_engulfing | Entry: 2025-01-30 at 1253.60, Exit: 2025-02-01 at 1269.95 (Target: 1278.67, Reason: max_window_exit), P/L: 16.35
bullish_engulfing | Entry: 2025-02-19 at 1226.95, Exit: 2025-02-21 at 1239.95 (Target: 1251.49, Reason: max_window_exit), P/L: 13.00
bearish_engulfing | Entry: 2025-03-25 at 1285.40, Exit: 2025-03-26 at 1269.00 (Target: 1259.69, Reason: min_window_exit), P/L: 16.40


In [27]:
plot_candlestick(data)